In [0]:
import dlt
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# -------------------------------
# Config
# -------------------------------
CATALOG = "dev_aoc_catalog"
SILVER_SCHEMA = "silver"
SILVER_TABLES = f"{CATALOG}.{SILVER_SCHEMA}"

# -------------------------------
# Gold Table DLT
# -------------------------------

@dlt.table(
    name="ga_dim_users",
    comment="Gold table for GA users dimension",
    partition_cols=["home_country"]
)
def ga_dim_users():
    # Step 1: Read Silver Table
    silver_df = dlt.read(f"{SILVER_TABLES}.ga_silver_web_users")
    
    # Step 2: Convert timestamps and map columns
    df = silver_df.select(
        F.col("user_id"),
        F.col("pseudo_id"),
        F.to_date("first_seen_time").alias("first_seen_date"),
        F.col("identified_user_flg").alias("known_user_flag"),
        F.col("home_country"),
        F.col("home_city"),
        F.col("preferred_device"),
        F.col("first_touch_medium"),
        F.col("first_touch_source"),
        F.col("first_touch_campaign"),
        F.col("user_property_name"),
        F.col("user_property_value"),
        F.col("is_active"),
        F.current_timestamp().alias("upsert_dttm")
    )
    
    # Step 3: Deduplicate by business key (user_id)
    # window_spec = Window.partitionBy("pseudo_id").orderBy(F.col("upsert_dttm").desc())
    # deduped_df = df.withColumn("row_num", F.row_number().over(window_spec)) \
    #                .filter(F.col("row_num") == 1) \
    #                .drop("row_num")
    
    # Step 4: Generate surrogate key (unique for gold)
    df = df.withColumn("user_sk", F.monotonically_increasing_id())
    
    # Step 5: Select final columns in gold order
    return df.select(
        "user_sk",
        "user_id",
        "pseudo_id",
        "first_seen_date",
        "known_user_flag",
        "home_country",
        "home_city",
        "preferred_device",
        "first_touch_medium",
        "first_touch_source",
        "first_touch_campaign",
        "user_property_name",
        "user_property_value",
        "upsert_dttm",
        "is_active"
    )


@dlt.table(
    name="ga_fct_sessions",
    comment="Gold fact table for GA sessions (dummy date_sk)",
    partition_cols=["session_start_dt"]
)
def ga_fct_sessions():
    # Step 1: Read Silver Sessions
    silver_sessions_df = dlt.read(f"{SILVER_TABLES}.ga_silver_web_sessions")
    
    # Step 2: Read Gold Users to join and get user_sk
    users_df = dlt.read("ga_dim_users").select("user_sk", "user_id")
    
    # Step 3: Aggregate engagement_time to get session_duration_sec
    # Adjust this if engagement_time is stored in a different column
    session_agg_df = silver_sessions_df.groupBy(
        "session_id", "user_id", "session_start_time",
        "device_category", "os", "browser",
        "traffic_source", "traffic_medium", "traffic_campaign",
        "is_engaged_session"
    ).agg(
        (F.sum("session_sequence")/1000).alias("session_duration_sec")  # dummy assumption
    )
    
    # Step 4: Join with user_sk
    session_with_user_df = session_agg_df.join(users_df, on="user_id", how="left")
    
    # Step 5: Create dummy date_sk (e.g., YYYYMMDD as int)
    session_with_date_df = session_with_user_df.withColumn(
        "session_start_dt", F.col("session_start_time").cast("date")
    ).withColumn(
        "date_sk", F.col("session_start_dt")  # same as session_start_dt
    )
    
    # Step 6: Generate surrogate key for session_sk
    final_df = session_with_date_df.withColumn("session_sk", F.monotonically_increasing_id())
    
    # Step 7: Add session_count and insert_dttm
    final_df = final_df.withColumn("session_count", F.lit(1)) \
                       .withColumn("insert_dttm", F.current_timestamp())
    
    # Step 8: Select final columns in gold order
    return final_df.select(
        "session_sk",
        "session_id",
        "user_sk",
        "date_sk",
        "session_start_dt",
        "session_duration_sec",
        "is_engaged_session",
        F.col("traffic_source").alias("utm_source_last"),
        F.col("traffic_medium").alias("utm_medium_last"),
        F.col("traffic_campaign").alias("utm_campaign_last"),
        "device_category",
        "os",
        "browser",
        "session_count",
        "insert_dttm"
    )

@dlt.table(
    name="ga_fct_submissions",
    comment="Gold fact table for GA form submissions",
    partition_cols=["form_submit_dt"]
)
def ga_fct_submissions():
    # Step 1: Read Silver Form Fills
    form_fills_df = dlt.read(f"{SILVER_TABLES}.ga_silver_form_fill")
    
    # Step 2: Read Gold Users to get user_sk
    users_df = dlt.read("ga_dim_users").select("user_sk", "pseudo_id")
    
    # Step 3: Join with users
    df_with_user = form_fills_df.join(
        users_df, 
        form_fills_df["user_id"] == users_df["pseudo_id"],
        how="left"
    ).drop("pseudo_id")
    
    # Step 4: Create date_sk and form_submit_dt
    df_with_date = df_with_user.withColumn(
        "form_submit_dt", F.col("form_submit_time").cast("timestamp")
    ).withColumn(
        "date_sk", F.col("form_submit_dt")
    )
    
    # Step 5: Generate surrogate key for fill_id
    df_with_sk = df_with_date.withColumn("fill_id", F.monotonically_increasing_id())
    
    # Step 6: Add form_submit_count and insert_dttm
    df_with_metadata = df_with_sk.withColumn("form_submit_count", F.lit(1)) \
                                 .withColumn("insert_dttm", F.current_timestamp())
    
    # Step 7: Deduplicate by user_sk keeping the latest form_submit_dt
    window_spec = Window.partitionBy("user_sk").orderBy(F.col("form_submit_dt").desc())
    deduped_df = df_with_metadata.withColumn("row_num", F.row_number().over(window_spec)) \
                                 .filter(F.col("row_num") == 1) \
                                 .drop("row_num")
    
    # Step 8: Select final columns
    return deduped_df.select(
        "fill_id",
        "session_id",
        "user_sk",
        "date_sk",
        "form_submit_dt",
        "form_id",
        "form_name",
        "form_type",
        "form_page_url",
        "form_submit_count",
        "insert_dttm"
    )

@dlt.table(
    name="ga_fct_purchases",
    comment="Gold fact table for GA purchases",
    partition_cols=["purchase_date"]
)
def ga_fct_purchases():
    # Step 1: Read Silver Purchases
    purchases_df = dlt.read(f"{SILVER_TABLES}.ga_silver_purchase")
    
    # Step 2: Read Gold Users to get user_sk
    users_df = dlt.read("ga_dim_users").select("user_sk", "user_id")
    
    # Step 3: Read Gold Sessions to get session_sk
    sessions_df = dlt.read("ga_fct_sessions").select("session_sk", "session_id", "user_sk")
    
    # Step 4: Join with users
    df_with_user = purchases_df.join(users_df, on="user_id", how="left")
    
    # Step 5: Join with sessions
    df_with_session = df_with_user.join(
        sessions_df,
        on=["session_id", "user_sk"],
        how="left"
    )
    
    # Step 6: Create dummy date_sk from purchase_date
    df_with_date = df_with_session.withColumn(
        "purchase_date", F.col("purchase_date").cast("date")
    ).withColumn(
        "date_sk", F.date_format(F.col("purchase_date"), "yyyyMMdd").cast("int")
    )
    
    # Step 7: Generate surrogate key for purchase_id
    final_df = df_with_date.withColumn("purchase_id", F.monotonically_increasing_id())
    
    # Step 8: Add insert_dttm
    final_df = final_df.withColumn("insert_dttm", F.current_timestamp())
    
    # Step 9: Select final columns in gold order
    return final_df.select(
        "purchase_id",
        "transaction_id",
        "session_sk",
        "user_sk",
        "date_sk",
        "purchase_date",
        "product_id",
        "product_name",
        "item_variant",
        "quantity",
        "price",
        "insert_dttm"
    )

@dlt.table(
    name="ga_fct_pageviews",
    comment="Gold fact table for GA pag eviews",
    partition_cols=["page_view_dt"]
)
def ga_fct_pageviews():
    # Step 1: Read Silver Page Views
    pageviews_df = dlt.read(f"{SILVER_TABLES}.ga_silver_page_views")
    
    # Step 2: Read Gold Users to get user_sk
    users_df = dlt.read("ga_dim_users").select("user_sk", "user_id")
    
    # Step 3: Join pageviews with users to get user_sk
    pageviews_with_user_df = pageviews_df.join(
        users_df, 
        pageviews_df.user_id == users_df.user_id, 
        how="left"
    ).drop(users_df.user_id)
    
    # Step 4: Create page_view_dt and date_sk from view_timestamp
    pageviews_with_date_df = pageviews_with_user_df.withColumn(
        "page_view_dt", F.col("view_timestamp").cast("timestamp")
    ).withColumn(
        "date_sk", F.date_format(F.col("page_view_dt"), "yyyyMMdd").cast("int")
    )
    
    # Step 5: Generate surrogate key for page_view_id
    final_df = pageviews_with_date_df.withColumn(
        "page_view_id", 
        F.hash(F.concat_ws("_", F.col("session_id"), F.col("view_timestamp"), F.col("page_url")))
    )
    
    # Step 6: Add view_count, page_category, and insert_dttm
    final_df = final_df.withColumn("view_count", F.lit(1)) \
                       .withColumn("page_category", F.lit(None).cast("string")) \
                       .withColumn("insert_dttm", F.current_timestamp())
    
    # Step 7: Rename engagement_time to engagement_time_sec if needed
    if "engagement_time" in final_df.columns:
        final_df = final_df.withColumnRenamed("engagement_time", "engagement_time_sec")
    
    # Step 8: Select final columns in gold order
    return final_df.select(
        "page_view_id",
        "session_id",      # Natural key instead of session_sk
        "user_sk",
        "date_sk",
        "page_view_dt",
        "page_url",
        "page_title",
        "page_category",
        "engagement_time_sec",
        "view_count",
        "insert_dttm"
    )